<a href="https://colab.research.google.com/github/dolmani38/similarly/blob/master/BERT_Sentence_similarity_0720.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jul 20 15:20:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
# 한국어 영화 리뷰를 Load.
ds = pd.read_csv("https://raw.githubusercontent.com/dolmani38/similarly/master/ratings_test.txt",sep='\t')
ds

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


##Sentence Transformers: Multilingual Sentence Embeddings using BERT / RoBERTa / XLM-RoBERTa & Co. with PyTorch

참조:https://github.com/UKPLab/sentence-transformers

In [3]:
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.3.0)


##영어 + 한국어 BERT 모델의 사용

#Extending Sentence Embeddings Models to New Languages

#Available Pre-trained Models


참조:https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/multilingual-models.md

In [4]:
from sentence_transformers import SentenceTransformer
# embedder download...
embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

In [5]:
#영화 리뷰의 내용을 corpus array에 저장한다.

corpus = []
for row in ds.iterrows():
    doc_id = row[1][0]
    doc_cont = str(row[1][1])
    corpus.append(doc_cont)

print('corpus size:',len(corpus))

corpus size: 50000


In [6]:
#하나의 sentence를 embedding해 본다.
#약 123ms 소요되며, 1024 차원의 vector를 생성한다.

from timeit import default_timer as timer
from datetime import timedelta
start = timer()

sentence_embedding = embedder.encode([corpus[4]],show_progress_bar=True)   

end = timer()

print("End... elapsed time : {}".format(timedelta(seconds=end-start)))
print('vector dim:',len(sentence_embedding[0]))
sentence_embedding


End... elapsed time : 0:00:00.297865
vector dim: 1024


[array([ 0.6906828 , -0.2728697 ,  0.541603  , ..., -0.58096015,
         0.6411979 , -0.16210863], dtype=float32)]

In [7]:
corpus[4]

'3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??'

In [8]:
# 50000개 전체 corpus를 embedding한다.

start = timer()
corpus_embeddings = embedder.encode(corpus,show_progress_bar=True) 
end = timer()

print("End... elapsed time : {}".format(timedelta(seconds=end-start)))


End... elapsed time : 0:22:39.508602


In [9]:
# 유사한 sentence를 찾는 함수의 정의
import scipy

def searchSentence(queries,Threshold_score=0.60, Topn=0):
    index = {}
    n = len(queries)
    query_embeddings = embedder.encode(queries)
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        
        for idx, distance in results[0:len(results) if Topn==0 else Topn]:
          score = 1-distance
          if not idx in index:
            index[idx] = 0.0
          index[idx] += float(score/n)

    return index

In [10]:
# Quary를 넣고 유사한 리뷰를 찾아 낸다.

query = ['개 쓰레기']
rank_count = 10
start = timer()
idx = searchSentence(query, Topn=rank_count)
end = timer()
print("End... elapsed time : {}".format(timedelta(seconds=end-start)))
ds.loc[list(idx.keys()),'score'] = list(idx.values())
ds.loc[list(idx.keys())]

End... elapsed time : 0:00:00.395806


,id,document,label,score
13165,7407403,개 쓰레기,0,1.000000
35236,3234260,개 쓰레기,0,1.000000
40103,5494444,개 쓰레기,0,1.000000
34785,9340926,쓰레기... 개 쓰레기,0,0.953830
40663,6087414,개 망작,0,0.780994
1564,7513006,개쩜,1,0.750314
10220,3127050,개쩜,1,0.750314
15209,801423,"개 졸 작,",0,0.746694
5201,6902301,쓰래기,0,0.742700
15439,7540167,쓰래기,0,0.742700


## Locality sensitive hashing를 이용한 유사 문장 탐색

참조:https://en.wikipedia.org/wiki/Locality-sensitive_hashing

참조:https://github.com/pixelogik/NearPy

In [11]:
# 이번에는 Local sensitive hash를 이용한다.
!pip install NearPy

In [12]:
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections
from nearpy.distances import CosineDistance
import numpy as np

# Dimension of our vector space
dimension = len(corpus_embeddings[0])

# Create a random binary hash
rbp = RandomBinaryProjections('rbp', 4)

# Create engine with pipeline configuration
engine = Engine(dimension, lshashes=[rbp],distance=CosineDistance())

In [13]:
for index in range(len(corpus_embeddings)):
    engine.store_vector(corpus_embeddings[index],index)

In [14]:

query = ['개 쓰레기']
query_embeddings = embedder.encode(query)
# Get nearest neighbours
start = timer()
N = engine.neighbours(query_embeddings[0])
end = timer()
print("End... elapsed time : {}".format(timedelta(seconds=end-start)))
pN = np.asarray(N)
pN.shape

ds.loc[list(pN[:,1]),'score'] = list(1-pN[:,2])
ds.loc[list(pN[:,1])]



End... elapsed time : 0:00:00.004383


,id,document,label,score
13165,7407403,개 쓰레기,0,1.000000
35236,3234260,개 쓰레기,0,1.000000
40103,5494444,개 쓰레기,0,1.000000
11363,10238777,여주 개암걸림 후반부터 개짜증,0,0.611821
26084,5882967,우울해지는영화 .. 개감동,1,0.598353
20875,9374253,개 지루함 개 잼없음 몰입도 꽝,0,0.560784
3181,1754960,개지루...,0,0.535112
38912,4939010,유지태가 아까운 개똥폼 잡는 깡패영화..,0,0.512046
17310,3865384,안타까움,0,0.511534
26696,7235423,아쉽다.,0,0.500452
